In [ ]:
from modelscope import snapshot_download
model_dir = snapshot_download('Qwen/Qwen3-0.6B', cache_dir="./")

In [2]:
!pip install modelscope

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 26.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [5]:
!pip install transformers==4.51.1

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 28.2 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 27.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0___6B")

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-0___6B"
)

In [9]:
# prepare the model input
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

thinking content: <think>
Okay, the user wants a short introduction to a large language model. Let me start by recalling what I know about them. LLMs, like GPT, are big language models trained on massive datasets. They can understand and generate text, which makes them useful for various tasks.

I should mention their ability to understand and generate text. Maybe include some examples like writing a story or answering questions. Also, highlight their applications in fields like science, tech, and creative industries. Keep it concise but informative. Avoid technical jargon to make it accessible. Check if there's anything else they might need, but the user didn't ask for that. Make sure the introduction flows well and covers the key points.
</think>
content: A large language model (LLM) is a type of artificial intelligence system designed to understand and generate human language. These models, such as GPT, are trained on vast datasets to comprehend complex texts, respond to questions, 

In [10]:
def generate_text(prompt, temperature=1.0, top_p=0.9, top_k=0, max_length=100, repetition_penalty=1.2):
    messages = [
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # conduct text completion
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=32768,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k if top_k > 0 else None,
        repetition_penalty=repetition_penalty,
        do_sample=True
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

    # parsing thinking content
    try:
        # rindex finding 151668 (</think>)
        index = len(output_ids) - output_ids[::-1].index(151668)
    except ValueError:
        index = 0

    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

    return content

In [11]:
generate_text("介绍一下人工智能对现代社会的影响")

'人工智能（Artificial Intelligence, AI）正深刻地重塑现代社会，并带来多维度的变化。以下是一些关键影响：\n\n---\n\n### **1. 经济与工作：效率革命与转型**\n- **自动化替代传统岗位**  \n  工业机器人、数据分析工具等取代了大量重复性劳动，减少了人为错误率并提高了生产效率。\n- **催生新职业和技术密集型行业**  \n  包括机器学习工程师、自动化工厂管理人员等新兴职位不断涌现，推动产业升级。\n- **全球化经济中的劳动力配置优化**  \n  全球化程度提高后，高技能人才向科技强国转移，形成以技术驱动的职业模型。\n\n---\n\n### **2. 教育：个性化学习的新时代**\n- **智能化教学平台普及**  \n  自动分析学生的进度与行为模式，提供定制化的教学内容，提升学习效果。\n- **虚拟教师与自适应学习系统**  \n  让在线课程更互动性强，打破地域限制的同时为弱势群体提供教育资源。\n- **终身学习意识的觉醒**  \n  随着知识更新速度加快，人们意识到持续学习的重要性。\n\n---\n\n### **3. 医疗健康：精准诊断与健康管理升级**\n- **医学影像诊断辅助**  \n  使用计算机视觉识别图像，帮助医生更快准确判断癌症或其他病症。\n- **远程医疗服务扩大至全球**  \n  移动端设备连接医院资源，让偏远地区患者获得同等质量的医疗服务。\n- **智能药物研发加速**  \n  大规模基因组数据库与神经网络算法降低制药成本与时间消耗。\n\n---\n\n### **4. 社会生活：便利性飞跃与隐私挑战**\n- **智能家居系统的互联整合**  \n  家庭物联网（IoT）使生活更加便捷，从控制家电到管理安全监控都实现无缝对接。\n- **自动驾驶与交通系统革新**  \n  可视化路径规划减少事故概率，但同时也引发关于法律责任及道德困境的问题。\n- **数据滥用风险加剧**  \n  快速积累的个人信息可能被用于商业操控，需建立完善的法律法规框架。\n\n---\n\n### **5. 环境可持续发展：能源与气候应对的关键力量**\n- **环境监测与气候预测建模**  \n  数据驱动的技术提升了灾害预警能力，支撑绿色金融项目。\n- **工业排放控

In [19]:
# 测试参数范围
param_ranges = {
    "temperature": [0.1, 0.5, 0.7, 1.0, 1.5, 2.0],
    "top_p": [0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99],
    "top_k": [0, 5, 10, 20, 50, 100],
    "repetition_penalty": [1.0, 1.2, 1.5, 2.0],
    "max_length": [50, 100, 200]
}

# 测试函数
def test_generation_parameters(prompt, num_tests=3):
    results = []
    baseline_output = generate_text(prompt, temperature=1.0, top_p=1.0, top_k=0, max_length=100)
    print(f"\n基准输出 (temperature=1.0, top_p=1.0, top_k=0):\n{baseline_output}\n")
    
    # 测试不同参数
    for param, values in param_ranges.items():
        for value in values:
            for i in range(num_tests):
                try:
                    # 设置默认参数
                    kwargs = {
                        "temperature": 1.0,
                        "top_p": 0.9,
                        "top_k": 50,
                        "repetition_penalty": 1.2,
                        "max_length": 100
                    }
                    kwargs[param] = value  # 覆盖当前测试参数
                    
                    # 生成文本
                    output = generate_text(prompt, **kwargs)
                    print(f"params is{kwargs} \n output is {output}")
                    # 收集结果
                    results.append({
                        "parameter": param,
                        "value": value,
                        "test_num": i+1,
                        "output": output
                    })
                    print(f"测试: {param}={value} | 输出: {output[:60]}...")
                
                except Exception as e:
                    print(f"错误: {param}={value} | {str(e)}")
    
    return pd.DataFrame(results)


ModuleNotFoundError: No module named 'pandas'

In [21]:
import pandas as pd


In [22]:
df = test_generation_parameters("介绍一下人工智能对现代社会的影响", 1)


基准输出 (temperature=1.0, top_p=1.0, top_k=0):
人工智能（Artificial Intelligence, AI）对现代社会产生了深远而深刻的变化，其影响力覆盖了经济、文化、教育、医疗等多个层面。以下是核心方面的具体情况：

### 1. **经济增长与技术创新**
   - **产业升级**：AI推动工业自动化和智能生产，提升了制造效率并降低了成本；关键行业如金融、制造业面临智能化改造。
   - **新兴产业崛起**：医疗、通信、金融服务等高度依赖AI的企业迅速成长，催生新职业和商业模式（如自动驾驶、智能客服）。

---

### 2. **就业市场调整**
   - **技能紧缩效应**：劳动密集型岗位减少，高技能人才需求激增，企业改革转向技能型 workforce。
   - **替代就业岗位**：机器取代重复任务，导致部分传统产业“消失”，引发社会公平议题及技能培训的压力。
   - **新兴工作形态**：全球范围内出现更多基于AI辅助创造的新职位（如数据分析工程师、算法设计师），但这种变革会加剧结构性矛盾。

---

### 3. **教育资源革新**
   - **终身学习导向**：在线课程平台加快知识普及，人工智能驱动的个性化学习系统改善了教育质量。
   - **人文教育融合**：AI促进了跨学科研究（如结合量子计算和基因工程），但也可能存在信息过载或兴趣失衡风险。

---

### 4. **健康与医疗服务**
   - **精准化医疗**：IBM Watson癌症筛查系统及药品推荐工具显著提高了疾病诊断准确率和治疗效果。
   - **心理健康援助**：NLP技术处理患者情绪评估，增强远程互助功能，缓解传统心理服务体系压力。

---

### 5. **社会治理生态**
   - **公共决策优化**：大数据分析助力政策制定（如政府利用AI预测突发事件）。
   - **法治理论更新**：面对法律系统被程序自动化的趋势，各国正通过立法保障人机协作权。

---

### 6. **环境保护与发展**
   - **能源节约潜力**：AI驱动的大规模供水/电力系统可在局部地区实现节能减排。
   - **危机预警能力**：卫星图像识别消除了灾害前兆漏报问题，生物测量工具进一步缩短病患康复时间。



In [23]:
df

,parameter,value,test_num,output
0,temperature,0.10,1,"人工智能（Artificial Intelligence, AI）作为21世纪最具革命性的科..."
1,temperature,0.50,1,"人工智能（Artificial Intelligence, AI）作为21世纪最具变革性的技..."
2,temperature,0.70,1,"人工智能（Artificial Intelligence, AI）作为21世纪最具革命性的技..."
3,temperature,1.00,1,"人工智能（Artificial Intelligence, AI）作为21世纪最具变革性的技..."
4,temperature,1.50,1,"人工智能（Artificial Intelligence, AI）正在重塑我们生活方式，深刻..."
5,temperature,2.00,1,在现代社会中，人工智能（AI）的发展正在深刻重塑产业格局、生产模式甚至生活方式。以下为几个关...
6,top_p,0.10,1,"人工智能（Artificial Intelligence, AI）作为21世纪最具革命性的科..."
7,top_p,0.30,1,"人工智能（Artificial Intelligence, AI）作为21世纪最具革命性的科..."
8,top_p,0.50,1,"人工智能（Artificial Intelligence, AI）作为21世纪最具革命性的科..."
9,top_p,0.70,1,"人工智能（Artificial Intelligence, AI）作为21世纪最具革命性的科..."


In [26]:
df.to_excel("test_result.xlsx")

In [25]:
!pip install openpyxl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://mirrors.aliyun.com/pypi/simple

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
